In [3]:
from gensim.models import Word2Vec
import gensim.downloader as api
from pathlib import Path
import os
import fintex_functions
import importlib
importlib.reload(fintex_functions)
from fintex_functions import analogy_function, most_similar_function, unmatched_function, word_embedding_2d_representation
from fintex_functions import single_word_embedding_2d_representation

In [1]:
import tensorflow as tf
# from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Dropout, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import RandomSearch
import numpy as np

import re
from keras.preprocessing.text import one_hot, Tokenizer
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
import tensorflow
from io import StringIO
import pandas as pd
import numpy as np 
import keras
import re
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from numpy import array
from keras.preprocessing.text import one_hot, Tokenizer 
from keras.utils import pad_sequences 
from keras.models import Sequential, Model
# from keras.layers.core import
from keras.layers import Flatten, Input, Concatenate, GlobalMaxPooling1D, Embedding, Conv1D, LSTM, GRU, MultiHeadAttention, Add, Bidirectional,  Activation, Dropout, Dense, GRU
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as K
import seaborn as sns
import nltk
from numpy import asarray
from numpy import zeros
from keras.layers import Conv1D
from keras.layers import LSTM
import json
from keras.optimizers import Adam
# from tensorflow.keras.utils import serialize_keras_object
from tensorflow.keras.utils import serialize_keras_object
from tensorflow.keras import utils
from tensorflow.keras.models import clone_model, Sequential
from tensorflow.keras.layers import Dense, Flatten
import joblib
from keras.models import load_model
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_auc_score
lemmatizer = WordNetLemmatizer()
text_tokenizer = nltk.RegexpTokenizer(r"\w+")
stop_words = set(stopwords.words('english'))
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet



C:\Users\benjo\AppData\Local\Temp\ipykernel_14796\4216099227.py:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\benjo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
import pandas as pd
import numpy as np
import seaborn as sns

## Load in training data

In [5]:
fin_news_1 = pd.read_csv('fin_news_optimized_6.csv')

In [ ]:
sns.countplot(x='labels', data=fin_news_1)

## Load text embedding

In [ ]:
embedding_folder_path = "C:\\Users\\benjo\\Final Project\\Assets\\Word embedings\\"
file_location = 'FinText_Word2Vec_CBOW\\Word_Embedding_2000_2015'

FinText_Word2Vec_cbow = Word2Vec.load(embedding_folder_path + file_location)

## Pre-processing data

In [ ]:
def preprocess_sentence(text_):

    # Remove links, mentions     
    # reduce all text to lower case
    text_ = text_.lower()

    #Remove mentions
    text_ = re.sub('@\S+', ' ', text_)
    
    #Remove any links
    text_ = re.sub('https?:\S+|http?:\S', ' ', text_)
    
    # Remove puntuations and numbers
    text_ = re.sub('[^a-zA-Z]', ' ', text_)

    # Single character removal 
    text_ = re.sub(r"\s+[a-zA-z]\s+", ' ', text_)

    # Remove multiple spaces
    text_ = re.sub(r'\s+', ' ', text_)
    
    # Lematize
#     tokens_result = tokenizer.tokenize(tweet_)
#     pos_tuple_list = nltk.pos_tag(tokens_result)
#     POS_TRANSLATED = 
#     POS_TRANS()
#     lemmatized_tokens = [lemmatizer.lemmatize(t) for t in tokens_result]

#     combined_sentence = " ".join(lemmatized_tokens)
#     pattern_ = re.compile(r'\b(' + r'|'.join(stop_words) + r')\b\s*')
#     tweet_ = pattern_.sub('', tweet_)

    return text_


def Xtrain_pre_processing(x_feature):
    feature = []
    text_ = list(x_feature)
    for txt in text_:
        sent = preprocess_sentence(txt)
#         sent = clean_text(txt)
        
#         extr_con_val = extract_concept_value_pairs(sent)
        feature.append(sent)
    return feature


In [6]:
X = Xtrain_pre_processing(fin_news_optimized_6['sentence'])
fin_news_optimized_6['labels'] = fin_news_optimized_6['labels'].astype(int)
y = fin_news_optimized_6['labels']

In [ ]:
X0_train, X0_test, y0_train, y0_test =  train_test_split(X, y, test_size = 0.25, random_state = 42)

## Preparing the Embedding Layer

In [ ]:
word_tokenizer = Tokenizer()

word_tokenizer.fit_on_texts(X0_train)

vocab_size = len(word_tokenizer.word_index) + 1
vocab_size

In [ ]:
#  This function prepares the embedding layer for our NN, it takes in the feature input and returns a tokenized array with padding 
def prep_embedding_layer(feature_, posit_of_padd_, max_padd_len):


    feature_ = word_tokenizer.texts_to_sequences(feature_)

    feature_ = pad_sequences(feature_, padding=posit_of_padd_, maxlen=max_padd_len, truncating='post')
    
    return feature_

In [ ]:
max_length_ = FinText_Word2Vec_cbow.wv['word'].shape[0]

In [ ]:
X0_train_emb = prep_embedding_layer(X0_train, 'post', max_length_)
X0_test_emb = prep_embedding_layer(X0_test, 'post', max_length_)

In [ ]:
y0_train_OHE =  tensorflow.keras.utils.to_categorical(y0_train, num_classes=3)
y0_test_OHE =  tensorflow.keras.utils.to_categorical(y0_test, num_classes=3)

In [ ]:
X0_train[0]

In [ ]:
print((X0_train_emb.shape, X0_test_emb.shape ))

## Loading Embeddings

In [ ]:
FinText_Word2Vec_cbow.wv['hi']

In [ ]:
len(FinText_Word2Vec_cbow.wv['amazon'])

Length should be == 300

In [ ]:
def initialize_FinText(vocabSize, maxLength, wordInd, word_emb_model ):
    matrix = np.zeros((vocabSize, maxLength))  
    for word, i in wordInd.items():
        if word in FinText_Word2Vec_cbow.wv: 
            vector_ = word_emb_model[word]
            if vector_ is not None:
                matrix[i] = vector_
            
    return matrix

In [ ]:
embedding_matrix = initialize_FinText(vocab_size, max_length_, word_tokenizer.word_index, FinText_Word2Vec_cbow.wv)

In [ ]:
embedding_matrix

In [ ]:
X0_train[0]

In [ ]:
X0_train_emb[0]

In [ ]:
if embedding_matrix.shape[1] == 300:
    pritn('True')
    embedding_dim = embedding_matrix.shape[1]

## Create Model

### Define Custom Metrics

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

### Derive balancing weight values
Due to the datasets unbalanced nature, we would need to add class weights in order to eliminate bias towards the class that holds the highest volumen of data

In [ ]:
type(y0_train)

In [ ]:
y0_train_emb = np.array(y0_train)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
class_weights = compute_class_weight('balanced', classes=np.unique(y0_train_emb), y=y0_train_emb)
class_weight_dict = dict(enumerate(class_weights))

In [ ]:
class_weight_dict

### Creating the attention layer

In [ ]:
class MyAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(MyAttention, self).__init__()
        self.units = units

        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

        attention_weights = tf.nn.softmax(self.V(score), axis=1)  # Use self.V as a layer

        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

    def get_config(self):
        config = super(MyAttention, self).get_config()
        config.update({'units': self.units})
        return config

### Create model (Viklund) test 

We just want to see if the model architecture is the same as the one that is designed in my computer

In [ ]:
# Add Embedding Layer
sequence_input = Input(shape=(X0_train_emb.shape[1],), dtype="int32")
embedded_sequences = Embedding(vocab_size, 
                               embedding_dim,
                               weights=[embedding_matrix],
                               trainable = False)(sequence_input)

# Add Bidirectional layer
bilstm = Bidirectional(LSTM(64, return_sequences=True), name="bi_lstm_0")(embedded_sequences)
(lstm, forward_h, forward_c, backward_h, backward_c) = Bidirectional(LSTM(32, return_sequences=True, return_state=True), name="bi_lstm_1")(bilstm)


# Concatenate RNN hidden states and apply attention
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
context_vector, attention_weights = MyAttention(10)(lstm, state_h)

# Add Dense Layers
dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.05)(dense1)
output = Dense(3, activation="softmax")(dropout)


# Create the Model object
Bi_ATT_2_model = Model(inputs=sequence_input, outputs=output)

# Plot Model
# keras.utils.plot_model(Bi_AT_model, show_shapes=True, dpi=90)
Bi_ATT_2_model.summary()

## Build Hyper Parameter Tuning Helper Fucntion

Split dataset AGAIN

In [ ]:
X1_train_emb, X1_val_emb, y1_train_OHE, y1_val_OHE =  train_test_split(X0_train_emb, y0_train_OHE, test_size = 0.25, random_state = 42)

In [7]:


def create_bi_att_2_model(vocab_size, embedding_matrix, params):
    # Add Embedding Layer
    sequence_input = Input(shape=(X0_train_emb.shape[1],), dtype="int32")
    embedded_sequences = Embedding(vocab_size, 
                                   embedding_matrix.shape[1],  # Use the embedding_matrix dimension
                                   weights=[embedding_matrix],
                                   trainable=False)(sequence_input)

    # Add Bidirectional layer
    lstm_layers = []
    for _ in range(params['num_lstm_layers']):
        bilstm = Bidirectional(LSTM(params['lstm_units'], return_sequences=True), name=f"bi_lstm_{_}")(embedded_sequences)
        lstm_layers.append(bilstm)
        embedded_sequences = bilstm  # Pass output as input to the next layer

    (lstm, forward_h, forward_c, backward_h, backward_c) = Bidirectional(LSTM(params['lstm_units'], return_sequences=True, return_state=True), name="bi_lstm_final")(embedded_sequences)

    # Concatenate RNN hidden states and apply attention
    state_h = Concatenate()([forward_h, backward_h])
    state_c = Concatenate()([forward_c, backward_c])
    context_vector, attention_weights = MyAttention(params['attention_units'])(lstm, state_h)

    # Add Dense Layers
    dense1 = Dense(params['units'], activation="relu")(context_vector)
    dropout = Dropout(params['dropout_rate'])(dense1)
    output = Dense(3, activation="softmax")(dropout)

    # Create the Model object
    model = Model(inputs=sequence_input, outputs=output)
    
    return model

def build_random_search_model(hp):
    params = {
        'num_lstm_layers': hp.Choice('num_lstm_layers', values=[1, 2, 3]),  # Number of LSTM layers
        'learning_rate': hp.Choice('learning_rate', values=[0.001, 0.01, 0.1]),
        'batch_size': hp.Choice('batch_size', values=[32, 64, 128]),
        'units': hp.Choice('units', values=[16, 32, 64]),
        'dropout_rate': hp.Float('dropout_rate', min_value=0.2, max_value=0.4, step=0.1),
        'lstm_units': hp.Choice('lstm_units', values=[32, 64, 128]),
        'attention_units': hp.Choice('attention_units', values=[8, 16, 32])
    }
    
    model = create_bi_att_2_model(vocab_size, embedding_matrix, params)
    optimizer = Adam(learning_rate=params['learning_rate'])
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc', keras.metrics.AUC(name='AUC'), f1_m,precision_m, recall_m])
    return model

def random_search(x_tr, y_tr, x_val, y_val, num_trials_, obj_, cw_,early_stopping_,checkpointer_):
    tuner = RandomSearch(
        build_random_search_model,
        objective=obj_,
        max_trials=num_trials_,## Loading Embeddings
        directory='random_search_logs',
        project_name = 'Bi_ATT_random_search'
    )

    tuner.search(x=x_tr, 
                 y=y_tr, 
                 epochs=15, 
                 validation_data=(x_val , y_val ),
                 callbacks=[early_stopping_, checkpointer_]
                 class_weight=cw_ )

    best_model = tuner.get_best_models(1)[0]
    return best_model

# Define the hyperparameter search space
param_grid = {
    'num_lstm_layers': [1, 2, 3],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [32, 64, 128],
    'units': [16, 32, 64],
    'dropout_rate': [0.2, 0.3, 0.4],
    'lstm_units': [32, 64, 128],
    'attention_units': [8, 16, 32]
}


#### Save model in folder

In [ ]:
lstm_nn_path = "C:\\Users\\benjo\\Final Project\\Saved_Bi_ATT_2_model"

monitor_acc = 'val_acc'
monitor_auc = 'val_AUC'
checkpointer = ModelCheckpoint(filepath=lstm_nn_path,
                               verbose=1,
                               monitor=monitor_auc,
                               mode='max',
                               save_best_only=True)

early_stopping = EarlyStopping(monitor=monitor_auc,
                               patience=5,
                               mode='max',
                               restore_best_weights=True)

### Begin Hyper-Param Tuning 

In [ ]:
best_model = random_search(X1_train_emb, y1_train_OHE, X1_val_emb, y1_val_OHE, num_trials_=10, obj_='val_AUC', cw_=class_weight_dict, early_stopping, checkpointer)
best_model.summary()